In [ ]:
if __name__=="__main__": # main function
    
    S0 = 100
    sigma = 0.20
    K = 100
    rf = 0.05    
    T = 1     # maturity
    
    # discrete setup
    N = 40  # steps
    dt = T/N
    
    #
    u_req = np.exp(sigma*np.sqrt(dt))
    d_req = 1/u_req
    q_req = (np.exp(rf*dt)-d_req)/(u_req-d_req)
    
    u = 1.032
    d= 1/u
    #根据 binomial tree --> u*d=1
    q = (np.exp(rf*dt)-d)/(u-d)
    
    # required stepup
    print('current setup =(u=%f, d=%f, q=%f)\n'%(u,d,q))
    print('required setup =(u=%f, d=%f, q=%f)\n'%(u_req, d_req,q_req))
    
    Optype = 1 # 1: call -1: put 

    parameters_ex1 = {'model':{'S0':S0, 'sigma':sigma, 'rf':rf, 'u':u, 'd':d, 'q':q},                 
                  'asset':{'K':K, 'optype': Optype},
                  'setup':{'T':T, 'N':N, 'dt': T/N}                
                 }
    #print(parameters_ex1)
    
    parameters_ex2 = {'model':{'S0':S0, 'sigma':sigma, 'rf':rf, 'u':u, 'd':d, 'q':q},                 
                  'asset':{'K':K, 'optype': Optype},
                  'setup':{'T':T, 'N':N, 'dt': T/N}
                 }# test 2
    
    parameters_ex3 = {'model':{'S0':S0, 'sigma':sigma, 'rf':rf, 'u':u, 'd':d, 'q':q},                 
                  'asset':{'K':K, 'optype': Optype},
                  'setup':{'T':T, 'N':N, 'dt': T/N}
                 }# test 3

In [ ]:
# 用于计算 到期日 ST 的 function:
def StockPriceTree(parameters):
    S0=parameters['model']['S0']
    u=paramters['model']['u']
    d=parameters['model']['d']
    N=paramters['setup']['N']
    M=N+1# ? what is M
    
    #initialization of stock price:
    ST=np.zeros(M)
    for i in range(M):
        ST=S0*u**(i)*d**(N-i)
        return ST
    

In [ ]:
#用于 构建 option 的payoff 的 function：
# 使用的 codes和 ttl 不一样：？？
def PayyoffTree(ST,parameters):
    K = parameters['asset']['K'] 
    option_type = parameters['asset']['optype'] 
    
    if option_type==1:
        payoffs=np.amximum(0,ST-K)
    else:
         payoffs=np.amximum(0,K-ST)
            
    return payoffs
# Ttl codes:
#  # option info
#     K = parameters['asset']['K'] 
#     option_type = parameters['asset']['optype'] 
    
#     payoffs = np.maximum(0, (STs- K) if option_type == 1 else (K-STs))
    
#     return payoffs

In [ ]:
# 使用 BS formula pricing 的结果：
def Black_ScholesPrice(parameters):

    S0 = parameters['model']['S0']
    sigma = parameters['model']['sigma']
    K = parameters['asset']['K'] 
    rf = parameters['model']['rf']
    T = parameters['setup']['T']    
    option_type = parameters['asset']['optype'] 
    
    DF = np.exp(-rf*T)
    
    d1 = (np.log(S0/K)+(rf+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 1:  # call
        option_price = S0 * norm.cdf(d1) - K * DF * norm.cdf(d2)
    elif option_type == -1: # put
        option_price = K * DF * norm.cdf(-d2) - S0 * norm.cdf(-d1) 
    
    return option_price

In [ ]:
# 使用 binomial tree  pricing 的结果：
# general idea:
# 1. 需要discount factor 2.每一步的概率 3.每一步的payoff

def BinomialTreePricing(payoffs,parameters):
    
    rf = parameters['model']['rf']
    T = parameters['setup']['T']       
    DF = np.exp(-rf*T)
    
    u = parameters['model']['u']
    d = parameters['model']['d']    
    
    qu = parameters['model']['q']
    qd = 1 - qu
    
    N = parameters['setup']['N']   
    # initialize 2个空的 matrix 用来装 每一步的 discounted
    # payoffs：
    discountpayoffs = np.zeros(N+1) 
    
    #"N+1"--> 从0开始到N
    
    
    # 用来装 每一步的概率：
    node_probs = np.zeros(N+1)
    
    for i in range(N+1):
        prob_i=comb(N,N-i)*qu**(N-i)*qd**i
        node_probs[i]=prob_i
        discountpayoffs[i]=DF*payoffs[i]*node_probs[i]
    price=np.sum(node_probs[i])
    
    return price,discountpayoffs,node_probs
        

In [17]:
#对比 B-S公式 和 BT公式 对于 定价的差别：
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import comb
from scipy.stats import norm
if __name__=="__main__": # main function
    
    S0 = 100
    sigma = 0.20
    K = 100
    rf = 0.05    
    T = 1     # maturity
    
    # discrete setup
    N = 40  # steps
    dt = T/N
    
    #
    u_req = np.exp(sigma*np.sqrt(dt))
    d_req = 1/u_req
    q_req = (np.exp(rf*dt)-d_req)/(u_req-d_req)
    
    u = 1.032
    d= 1/u
    q = (np.exp(rf*dt)-d)/(u-d)
    
    # required stepup
    print('current setup =(u=%f, d=%f, q=%f)\n'%(u,d,q))
    print('required setup =(u=%f, d=%f, q=%f)\n'%(u_req, d_req,q_req))
    
    Optype = 1 # 1: call -1: put 

    parameters_ex1 = {'model':{'S0':S0, 'sigma':sigma, 'rf':rf, 'u':u, 'd':d, 'q':q},                 
                  'asset':{'K':K, 'optype': Optype},
                  'setup':{'T':T, 'N':N, 'dt': T/N}                
                 }
    
    
    
    
def StockPriceTree(parameters):

    # initial states
    S0 = parameters['model']['S0']
    
    # parameters
    u = parameters['model']['u']
    d = parameters['model']['d']    
    
    # total nodes
    N = parameters['setup']['N']
    M = N + 1  # starting from t=0
    
    # stock prices at maturity
    STs = np.zeros(M)
    for i in range(M):
        STs[i] = S0*u**(N-i)*d**i
        
    return STs

def PayoffTree(STs, parameters):
    
    # option info
    K = parameters['asset']['K'] 
    option_type = parameters['asset']['optype'] 
    
    payoffs = np.maximum(0, (STs- K) if option_type == 1 else (K-STs))
    
    return payoffs

def BinomialTreePrice(payoffs, parameters):
    # 两个 parameters
    # calcuate discounted payoff at each node at maturity
    rf = parameters['model']['rf']
    T = parameters['setup']['T']       
    DF = np.exp(-rf*T)
    
    u = parameters['model']['u']
    d = parameters['model']['d']    
    
    qu = parameters['model']['q']
    qd = 1 - qu
    
    N = parameters['setup']['N']     
    discountpayoffs = np.zeros(N+1)
    node_probs = np.zeros(N+1)
    
    for i in range(N+1): 
        prob_i = comb(N, N-i)*qu**(N-i)*qd**i
        node_probs[i] = prob_i
        discountpayoffs[i] = DF*payoffs[i]*node_probs[i]
        # 这里的"payoffs"是 payoff tree的返回值
        #也是 该 方程 需要 2个 parameters的原因
    price = np.sum(discountpayoffs)
    
    return price, discountpayoffs, node_probs  

def Black_ScholesPrice(parameters):

    S0 = parameters['model']['S0']
    sigma = parameters['model']['sigma']
    K = parameters['asset']['K'] 
    rf = parameters['model']['rf']
    T = parameters['setup']['T']    
    option_type = parameters['asset']['optype'] 
    
    DF = np.exp(-rf*T)
    
    d1 = (np.log(S0/K)+(rf+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 1:  # call
        option_price = S0 * norm.cdf(d1) - K * DF * norm.cdf(d2)
    elif option_type == -1: # put
        option_price = K * DF * norm.cdf(-d2) - S0 * norm.cdf(-d1) 
    
    return option_price


def BinomialTreevsBS(parameters):
    # 1. stock tree
    stocks = StockPriceTree(parameters)
    stocks_df = pd.DataFrame(stocks, index =list(range(0,parameters['setup']['N']+1)), columns=['stockprice'])
    
    # 2. option payoffs
    payoffs = PayoffTree(stocks, parameters)
    payoffs_df = pd.DataFrame(payoffs, index =list(range(0,parameters['setup']['N']+1)), columns=['payoffs'])
    
    # 3. price 
    [option_price_BT, discountpayoffs, node_probs] = BinomialTreePrice(payoffs, parameters)   
    dispayoffs_df = pd.DataFrame(discountpayoffs, index =list(range(0,parameters['setup']['N']+1)), columns=['disc_payoffs'])
    node_probs_df = pd.DataFrame(node_probs, index =list(range(0,parameters['setup']['N']+1)), columns=['node_probs'])
    
    # output & save files    
    binomial_all = pd.concat([stocks_df, payoffs_df, dispayoffs_df,node_probs_df],axis=1)
    
    print(binomial_all)  
    print("======================pricing by BinomialTree:")
    print(option_price_BT)
    option_price_BS = Black_ScholesPrice(parameters)
    print('Price_BT= %f vs Price_BS =%f\n'%(option_price_BT, option_price_BS))
    
    
#     binomial_all.to_excel(r"binomial_all.xlsx")
BinomialTreevsBS(parameters_ex1)

current setup =(u=1.032000, d=0.968992, q=0.511977)

required setup =(u=1.032128, d=0.968872, q=0.511868)

    stockprice     payoffs  disc_payoffs    node_probs
0   352.523353  252.523353  5.631346e-10  2.344366e-12
1   331.000384  231.000384  1.964142e-08  8.938711e-11
2   310.791478  210.791478  3.331481e-07  1.661495e-09
3   291.816408  191.816408  3.660343e-06  2.006092e-08
4   273.999843  173.999843  2.927629e-05  1.768813e-07
5   257.271050  157.271050  1.816093e-04  1.213959e-06
6   241.563617  141.563617  9.089648e-04  6.750099e-06
7   226.815187  126.815187  3.769963e-03  3.125219e-05
8   212.967207  112.967207  1.320479e-02  1.228836e-04
9   199.964702   99.964702  3.960252e-02  4.164768e-04
10  187.756053   87.756053  1.027317e-01  1.230671e-03
11  176.292790   76.292790  2.321821e-01  3.199337e-03
12  165.529406   65.529406  4.593966e-01  7.369979e-03
13  155.423170   55.423170  7.977136e-01  1.513110e-02
14  145.933960   45.933960  1.215387e+00  2.781606e-02
15  137.02410